In [1]:
!git clone https://github.com/AhsanBaidar/MuLA_GAN.git

Cloning into 'MuLA_GAN'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 125 (delta 30), reused 78 (delta 15), pack-reused 28 (from 1)
Receiving objects: 100% (125/125), 144.94 MiB | 44.04 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [2]:
%cd MuLA_GAN

/content/MuLA_GAN


In [5]:
# --- IMPORTANT: Fix this path if your 'Dataset' folder is not in the main 'My Drive' ---
gdrive_folder_path = "/content/drive/My Drive/Dataset"
# ------------------------------------------------------------------------------------

# Remove the link if it already exists (to avoid errors)
!rm -f ./Dataset

# Create the symbolic link
!ln -s "{gdrive_folder_path}" ./Dataset

print("Symbolic link created!")
print("The folder './Dataset' now points to your Google Drive folder.")

# Verify the link points correctly and the test folder exists within it
!ls -l ./Dataset/test/testA/

Symbolic link created!
The folder './Dataset' now points to your Google Drive folder.
total 63888
-rw------- 1 root root  518588 Jun  6  2023 102_img_.png
-rw------- 1 root root  224341 Jun  6  2023 11064.png
-rw------- 1 root root  460841 Jun  6  2023 12348.png
-rw------- 1 root root  893016 Jun  6  2023 12445.png
-rw------- 1 root root  481714 Jun  6  2023 150_img_.png
-rw------- 1 root root  585348 Jun  6  2023 158_img_.png
-rw------- 1 root root  653963 Jun  6  2023 163_img_.png
-rw------- 1 root root  477809 Jun  6  2023 17_img_.png
-rw------- 1 root root  605611 Jun  6  2023 192_img_.png
-rw------- 1 root root 1298287 Jun  6  2023 19_img_.png
-rw------- 1 root root  596031 Jun  6  2023 208_img_.png
-rw------- 1 root root  599950 Jun  6  2023 211_img_.png
-rw------- 1 root root  633720 Jun  6  2023 212_img_.png
-rw------- 1 root root  195168 Jun  6  2023 263_img_.png
-rw------- 1 root root 1165134 Jun  6  2023 2701.png
-rw------- 1 root root   92957 Jun  6  2023 282_img_.png
-rw--

In [6]:
!python test.py --weights_path "/content/drive/My Drive/generator_300.pth"

Loaded model from /content/drive/My Drive/generator_300.pth
Tested: ./Dataset/test/testA/102_img_.png
Tested: ./Dataset/test/testA/11064.png
Tested: ./Dataset/test/testA/12348.png
Tested: ./Dataset/test/testA/12445.png
Tested: ./Dataset/test/testA/150_img_.png
Tested: ./Dataset/test/testA/158_img_.png
Tested: ./Dataset/test/testA/163_img_.png
Tested: ./Dataset/test/testA/17_img_.png
Tested: ./Dataset/test/testA/192_img_.png
Tested: ./Dataset/test/testA/19_img_.png
Tested: ./Dataset/test/testA/208_img_.png
Tested: ./Dataset/test/testA/211_img_.png
Tested: ./Dataset/test/testA/212_img_.png
Tested: ./Dataset/test/testA/263_img_.png
Tested: ./Dataset/test/testA/2701.png
Tested: ./Dataset/test/testA/282_img_.png
Tested: ./Dataset/test/testA/294_img_.png
Tested: ./Dataset/test/testA/316_img_.png
Tested: ./Dataset/test/testA/320_img_.png
Tested: ./Dataset/test/testA/330_img_.png
Tested: ./Dataset/test/testA/337_img_.png
Tested: ./Dataset/test/testA/338_img_.png
Tested: ./Dataset/test/testA/35

In [7]:
%%writefile Evaluation/measure_ssim_psnr.py
"""
# > Script for measuring quantitative performances in terms of
#   - Structural Similarity Metric (SSIM)
#   - Peak Signal to Noise Ratio (PSNR)
# > Maintainer: https://github.com/xahidbuffon
"""
## python libs
import numpy as np
from PIL import Image
from glob import glob
from os.path import join, exists # Added exists
from ntpath import basename
import os
## local libs
# Make sure imqual_utils is importable
try:
    # Assumes running from MuLA_GAN root
    from Evaluation.imqual_utils import getSSIM, getPSNR
except ImportError:
    # Try importing from current directory if run from Evaluation/
    try:
         from imqual_utils import getSSIM, getPSNR
    except ImportError as e:
        print(f"Error importing imqual_utils: {e}")
        print("Please ensure you are running this script from the MuLA_GAN root directory or that Evaluation is in your Python path.")
        exit()


## compares avg ssim and psnr
def SSIMs_PSNRs(gtr_dir, gen_dir, im_res=(256, 256)):
    """
        - gtr_dir contain ground-truths
        - gen_dir contain generated images
    """
    gtr_paths = sorted(glob(join(gtr_dir, "*.*")))
    # Don't sort gen_paths here, look them up based on gtr_paths
    # gen_paths_all = sorted(glob(join(gen_dir, "*.*")))

    print(f"Found {len(gtr_paths)} potential ground truth images in: {gtr_dir}")
    print(f"Looking for corresponding generated images in: {gen_dir}")

    ssims, psnrs = [], []
    processed_count = 0

    if not gtr_paths:
        print(f"Error: No image files found in ground truth directory: {gtr_dir}")
        return np.array([]), np.array([])

    for gtr_path in gtr_paths:
        # Get base filename without extension
        gtr_f_base = basename(gtr_path).split('.')[0]
        # Construct expected generated file path (try common extensions)
        gen_path_expected_png = join(gen_dir, gtr_f_base + ".png")
        gen_path_expected_jpg = join(gen_dir, gtr_f_base + ".jpg")
        gen_path_expected_jpeg = join(gen_dir, gtr_f_base + ".jpeg")

        gen_path = None
        if exists(gen_path_expected_png):
            gen_path = gen_path_expected_png
        elif exists(gen_path_expected_jpg):
            gen_path = gen_path_expected_jpg
        elif exists(gen_path_expected_jpeg):
            gen_path = gen_path_expected_jpeg
        # Add more extensions if needed

        if gen_path: # Check if a corresponding file was found
            processed_count += 1
            # print(f"Processing pair: GT={gtr_path}, GEN={gen_path}") # Debug
            try:
                r_im = Image.open(gtr_path).resize(im_res)
                g_im = Image.open(gen_path).resize(im_res)

                # Ensure images are RGB before SSIM
                if r_im.mode != 'RGB': r_im = r_im.convert('RGB')
                if g_im.mode != 'RGB': g_im = g_im.convert('RGB')

                # get ssim on RGB channels
                ssim_val = getSSIM(np.array(r_im), np.array(g_im))
                if np.isfinite(ssim_val): ssims.append(ssim_val)

                # get psnr on L channel (grayscale)
                r_im_L = r_im.convert("L")
                g_im_L = g_im.convert("L")
                psnr_val = getPSNR(np.array(r_im_L), np.array(g_im_L))
                if np.isfinite(psnr_val): psnrs.append(psnr_val)

            except Exception as e:
                 print(f"Error processing {basename(gtr_path)} / {basename(gen_path)}: {e}")
        # else:
            # print(f"Skipping {basename(gtr_path)}: Corresponding generated file not found.") # Debug


    if processed_count == 0:
        print("\nError: No matching image pairs found between the two directories!")
        print("Please check paths and ensure filenames (without extension) are identical.")
        return np.array([]), np.array([]) # Return empty arrays

    return np.array(ssims), np.array(psnrs)


# --- Define YOUR paths here ---
# These paths assume you ran test.py and it saved to './output/'
# And that you have the original UIEB test data unzipped via symbolic link
gtr_dir = "./Dataset/test/testB/"  # Path to ground truth test images
gen_dir = "./output/"              # Path to your generated test images
# -----------------------------


### compute SSIM and PSNR
SSIM_measures, PSNR_measures = SSIMs_PSNRs(gtr_dir, gen_dir)

# Check if any valid results were returned before calculating mean/std
if len(SSIM_measures) > 0:
    print ("\n--- Results ---")
    print ("SSIM on {0} matched samples".format(len(SSIM_measures)))
    print ("Mean: {0:.4f} std: {1:.4f}".format(np.mean(SSIM_measures), np.std(SSIM_measures)))
else:
    print("\n--- No valid SSIM scores calculated. ---")


if len(PSNR_measures) > 0:
    print ("\nPSNR on {0} matched samples".format(len(PSNR_measures)))
    print ("Mean: {0:.2f} std: {1:.2f}".format(np.mean(PSNR_measures), np.std(PSNR_measures)))
else:
     print("\n--- No valid PSNR scores calculated. ---")

if len(SSIM_measures) == 0 and len(PSNR_measures) == 0:
    print ("\n--- No results calculated. Check paths, filenames, and potential errors during processing. ---")

Overwriting Evaluation/measure_ssim_psnr.py


In [8]:
!python Evaluation/measure_ssim_psnr.py

Found 90 potential ground truth images in: ./Dataset/test/testB/
Looking for corresponding generated images in: ./output/

--- Results ---
SSIM on 90 matched samples
Mean: 0.8099 std: 0.1171

PSNR on 90 matched samples
Mean: 22.98 std: 3.73


In [9]:
%%writefile Evaluation/measure_uiqm.py
"""
# > Script for measuring quantitative performances in terms of UIQM
# > Maintainer: https://github.com/xahidbuffon
"""
## python libs
import numpy as np
from PIL import Image, ImageOps
from glob import glob
from os.path import join
from ntpath import basename
import os # Added os import
## local libs
# Make sure uqim_utils is importable
try:
    # Assumes running from MuLA_GAN root
    from Evaluation.uqim_utils import getUIQM
except ImportError:
    # Try importing from current directory if run from Evaluation/
    try:
         from uqim_utils import getUIQM
    except ImportError as e:
        print(f"Error importing uqim_utils: {e}")
        print("Please ensure you are running this script from the MuLA_GAN root directory or that Evaluation is in your Python path.")
        exit()


def measure_UIQMs(dir_name, im_res=(256, 256)):
    paths = sorted(glob(join(dir_name, "*.*")))
    print(f"Found {len(paths)} files in: {dir_name}") # Debug print
    uqims = []
    i=0
    if not paths:
        print(f"Error: No images found in directory: {dir_name}")
        return np.array([])

    for img_path in paths:
        # print(f"Processing image {i+1}/{len(paths)}: {basename(img_path)}") # Debug print
        i=i+1
        try:
            im = Image.open(img_path).resize(im_res)
            # Ensure image is RGB for UIQM calculation
            if im.mode != 'RGB': im = im.convert('RGB')
            im_array = np.array(im)
            # print(im_array.shape) # Debug print shape
            uiqm = getUIQM(im_array)
            if np.isfinite(uiqm): # Check for NaN/Inf
                 uqims.append(uiqm)
            # else:
                # print(f"  -> Warning: UIQM calculation resulted in NaN/Inf for {basename(img_path)}")

        except Exception as e:
             print(f"Error processing {basename(img_path)}: {e}")

    if not uqims:
        print("\nError: UIQM calculation failed for all images.")
        return np.array([])

    return np.array(uqims)

# --- Define YOUR path here ---
# This path assumes you ran test.py and it saved to './output/'
gen_dir = "./output/"
# -----------------------------

# UIQMs of the enhanceded output images
gen_uqims = measure_UIQMs(gen_dir)

if len(gen_uqims) > 0: # Only print if results were calculated
    print ("\n--- Results ---")
    print ("UIQM on {0} samples".format(len(gen_uqims)))
    print ("Mean: {0:.4f} std: {1:.4f}".format(np.mean(gen_uqims), np.std(gen_uqims)))
else:
     print ("\n--- No UIQM results calculated. Check path and logs. ---")

Overwriting Evaluation/measure_uiqm.py


In [10]:
!python Evaluation/measure_uiqm.py

Found 90 files in: ./output/

--- Results ---
UIQM on 90 samples
Mean: 3.1258 std: 0.2882
